### Load the data

In [17]:
# curl is preinstall in macos
!curl -O https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  6299k      0 --:--:-- --:--:-- --:--:-- 6332k


In [18]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

## Encode data into training and validation sets

In [22]:
vocab = sorted(list(set(text)))
print(vocab)
stoi = {s: i for i, s in enumerate(vocab)}
itos = {i: s for i, s in enumerate(vocab)}
print('Total unique characters:', len(vocab))

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Total unique characters: 65


In [40]:
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda s: ''.join([itos[ch] for ch in s])

decode(encode('hello'))

'hello'

In [75]:
n = int(0.9*len(text))

data = encode(text)

import torch
data = torch.tensor(data, dtype=torch.long)
print(data.shape, data.dtype)

train_data = data[:n]
val_data = data[n:]

torch.Size([1115394]) torch.int64


In [ ]:
block_size = 8

def examples(arr):
    for num in range(block_size+1):
        print(arr[:num+1] ,':', arr[num+1])

examples(train_data[:8])

tensor([18]) : tensor(47)
tensor([18, 47]) : tensor(56)
tensor([18, 47, 56]) : tensor(57)
tensor([18, 47, 56, 57]) : tensor(58)
tensor([18, 47, 56, 57, 58]) : tensor(1)
tensor([18, 47, 56, 57, 58,  1]) : tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) : tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) : tensor(58)


In [84]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split == 'training' else val_data

    # generate a small batch of data of inputs x and targets y
    ix = torch.randint(0, len(data)-block_size, (batch_size, ))
    x = torch.stack([data[ind:ind+block_size] for ind in ix])
    y = torch.stack([data[ind+block_size] for ind in ix])
    return x, y

Xtr, Ytr = get_batch('training')
Xval, Yval = get_batch('val')